In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from engine.solver import Trainer
from Data.build_dataloader import build_dataloader, build_dataloader_cond
from Utils.io_utils import load_yaml_config, instantiate_from_config
from Models.interpretable_diffusion.model_utils import unnormalize_to_zero_to_one
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [ ]:
data = 'energy'

In [ ]:
class Args_Example:
    def __init__(self) -> None:
        self.config_path = f'./Config/{data}.yaml'
        self.save_dir = './toy_exp'
        self.gpu = 3
        os.makedirs(self.save_dir, exist_ok=True)

args =  Args_Example()
configs = load_yaml_config(args.config_path)
device = torch.device(f'cuda:{args.gpu}' if torch.cuda.is_available() else 'cpu')

dl_info = build_dataloader(configs, args)
model = instantiate_from_config(configs['model']).to(device)
trainer = Trainer(config=configs, args=args, model=model, dataloader=dl_info)

In [ ]:
trainer.train()

In [ ]:
class Args_Example:
    def __init__(self) -> None:
        self.name = 'energy'
        self.gpu = 3
        self.config_path = f'./Config/{self.name}.yaml'
        self.save_dir = './toy_exp'
        self.mode = 'infill'
        self.missing_ratio = 0.2
        self.milestone = 10
        os.makedirs(self.save_dir, exist_ok=True)

args =  Args_Example()
configs = load_yaml_config(args.config_path)
device = torch.device(f'cuda:{args.gpu}' if torch.cuda.is_available() else 'cpu')

dl_info = build_dataloader_cond(configs, args)
model = instantiate_from_config(configs['model']).to(device)
trainer = Trainer(config=configs, args=args, model=model, dataloader=dl_info)

trainer.load(args.milestone)

In [ ]:
dataloader, dataset = dl_info['dataloader'], dl_info['dataset']
coef = configs['dataloader']['test_dataset']['coefficient']
stepsize = configs['dataloader']['test_dataset']['step_size']
sampling_steps = configs['dataloader']['test_dataset']['sampling_steps']
seq_length, feature_dim = dataset.window, dataset.var_num
samples, reals, masks = trainer.restore(dataloader, [seq_length, feature_dim], coef, stepsize, sampling_steps)

if dataset.auto_norm:
    samples = unnormalize_to_zero_to_one(samples)
    reals  = unnormalize_to_zero_to_one(reals)
np.save(os.path.join(args.save_dir, f'ddpm_{args.mode}_{args.name}.npy'), samples)
np.save(os.path.join(args.save_dir, f'real_{args.mode}_{args.name}.npy'), reals)
np.save(os.path.join(args.save_dir, f'mask_{args.mode}_{args.name}.npy'), masks)

if args.name == 'sines':
    rescaled_reals = reals
    rescaled_samples = samples
else:
    print(reals.reshape(-1, reals.shape[-1]).shape)
    rescaled_reals = dl_info['dataset'].scaler.inverse_transform(reals.reshape(-1, reals.shape[-1])).reshape(reals.shape)
    rescaled_samples = dl_info['dataset'].scaler.inverse_transform(samples.reshape(-1, samples.shape[-1])).reshape(samples.shape)

masks = masks.astype(bool)
mse = mean_squared_error(rescaled_samples[~masks], rescaled_reals[~masks])
print(rescaled_reals.min(), rescaled_reals.shape)
print(f'Now with {args.missing_ratio} unobserved: {mse}')
mse = mean_squared_error(samples[~masks], reals[~masks])
print(f'Now with {args.missing_ratio} unobserved: {mse}')

In [ ]:
masks = masks.astype(bool)
mse = mean_squared_error(rescaled_samples[~masks], rescaled_reals[~masks])
print(rescaled_reals.min(), rescaled_reals.shape)
print(f'Now with {args.missing_ratio} unobserved: {mse}')
mse = mean_squared_error(samples[~masks], reals[~masks])
print(f'Now with {args.missing_ratio} unobserved: {mse}')

## Ploting the results

In [ ]:
import os
import warnings


warnings.filterwarnings("ignore")
if 'run' in os.getcwd():
    os.chdir('..')
    
os.environ["CUDA_VISIBLE_DEVICES"] = "3" # Possible GPUS

In [ ]:
LENGTH = 48

OPTS = f"model.params.seq_length {LENGTH} dataloader.train_dataset.params.window {LENGTH} dataloader.test_dataset.params.window {LENGTH} model.params.d_model {4*LENGTH}"

In [ ]:
SAMPLE = 1 # unconditional
MODE = 'infill'
NAME = 'energy'
MISSING = 0.5
print(f'----------------------------{NAME}-----------------------------------')
print('---Train---')
!CUDA_VISIBLE_DEVICES=3 python main.py --name {NAME} --sample=1 --mode {MODE} --train  {OPTS}
print('---Eval---')
!CUDA_VISIBLE_DEVICES=3 python main.py --name {NAME} --sample=1 --mode {MODE}  --missing_ratio {MISSING} {OPTS}

In [ ]:
plt.rcParams["font.size"] = 12
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 8))

real = np.load(os.path.join(args.save_dir, f'real_{args.mode}_{args.name}.npy'))
synthetic = np.load(os.path.join(args.save_dir, f'ddpm_{args.mode}_{args.name}.npy'))
mask = np.load(os.path.join(args.save_dir, f'mask_{args.mode}_{args.name}.npy'))


for i in range(10):
    plt.plot(real[i,:,0],linestyle='dashed')
    plt.plot(synthetic[i,:,0], c='r')

    plt.scatter(np.arange(0,24,1),mask[i,:,0]*real[i,:,0])
    plt.show()
print("*"*50)

In [ ]:
SAMPLE = 1 # unconditional
MODE = 'infill'
NAME = 'mujoco'
MISSING = 0.5
print(f'----------------------------{NAME}-----------------------------------')
print('---Train---')
!CUDA_VISIBLE_DEVICES=3 python main.py --name {NAME} --sample=1 --mode {MODE} --train  {OPTS}
print('---Eval---')
!CUDA_VISIBLE_DEVICES=3 python main.py --name {NAME} --sample=1 --mode {MODE}  --missing_ratio {MISSING} {OPTS}

In [ ]:
plt.rcParams["font.size"] = 12
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 8))

real = np.load(os.path.join(args.save_dir, f'real_{args.mode}_{args.name}.npy'))
synthetic = np.load(os.path.join(args.save_dir, f'ddpm_{args.mode}_{args.name}.npy'))
mask = np.load(os.path.join(args.save_dir, f'mask_{args.mode}_{args.name}.npy'))


for i in range(10):
    plt.plot(real[i,:,0],linestyle='dashed')
    plt.plot(synthetic[i,:,0], c='r')

    plt.scatter(np.arange(0,24,1),mask[i,:,0]*real[i,:,0])
    plt.show()
print("*"*50)